### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Eczema/GSE11223'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Assume gene expression data is available from the description

# Check availability and data type conversion for 'Eczema'
trait_row = 50  # 'other illnesses' contains 'Eczema' among other illnesses
def convert_trait(value):
    val = value.split(": ")[1].strip().lower()
    return 1 if "eczema" in val else 0

# Check availability and data type conversion for 'age'
age_row = 2  # 'birth date' can be used to infer age
from datetime import datetime

def convert_age(value):
    val = value.split(": ")[1].strip()
    try:
        birth_date = datetime.strptime(val, "%m/%d/%y")
        age = (datetime.now() - birth_date).days // 365  # Approximate age in years
        # Consider realistic age range
        if 0 < age < 120:
            return age
        else:
            return None
    except ValueError:
        return None

# Check availability and data type conversion for 'gender'
gender_row = None  # Gender information is not available
def convert_gender(value):
    val = value.split(": ")[1].strip().lower()
    if val in ["female", "f", "woman"]:
        return 0
    elif val in ["male", "m", "man"]:
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE11223', './preprocessed/Eczema/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical data if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Eczema', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Eczema/trait_data/GSE11223.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
if requires_gene_mapping:
    # 1. Decide which columns in gene annotation data store identifiers and gene symbols.
    identifier_key = 'SPOT_ID'
    gene_symbol_key = 'GENE_SYMBOL'

    # 2. Get the dataframe storing the mapping between probe IDs and genes
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # Check if the gene_mapping_df contains the expected columns
    print("Gene Mapping DataFrame Columns:", gene_mapping_df.columns)

    if 'ID' in gene_mapping_df.columns:
        # 3. Apply the mapping to get the gene expression dataframe
        gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
        print("Gene Mapping successfully applied")
    else:
        print("Error: The 'ID' column was not found in the gene mapping dataframe.")


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Eczema/gene_data/GSE11223.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Eczema')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE11223', './preprocessed/Eczema/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Eczema/GSE11223.csv'
    unbiased_merged_data.to_csv(csv_path)
